# TFIDF Only Solution

A Work in Progress...

An beginner friendly attempt to understand TFIDF by removing everything else and building a simple notebook.

## Other Focused Solutions

- [Shopee Word2Vect](https://www.kaggle.com/mmellinger66/shopeeword2vec)

## References

- https://www.kaggle.com/parthdhameliya77/pytorch-efficientnet-b3-image-tfidf-inference
- https://www.kaggle.com/cdeotte/part-2-rapids-tfidfvectorizer-cv-0-700#Use-Text-Embeddings

In [ ]:
import pandas as pd
import numpy as np

import gc
import cudf
import cuml
import cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors

In [ ]:
df = pd.read_csv('../input/shopee-product-matching/test.csv')
# df = pd.read_csv('../input/shopee-product-matching/train.csv')
df_cu = cudf.DataFrame(df)


# Using Text with Tfidf


In [ ]:
def get_text_predictions(df, max_features = 25_000):
    
    model = TfidfVectorizer(stop_words = 'english', binary = True, max_features = max_features)
    text_embeddings = model.fit_transform(df_cu['title']).toarray()
    preds = []
    CHUNK = 1024*4

    print('Finding similar titles...')
    CTS = len(df)//CHUNK
    if len(df)%CHUNK!=0: CTS += 1
    for j in range( CTS ):

        a = j*CHUNK
        b = (j+1)*CHUNK
        b = min(b,len(df))
        print('chunk',a,'to',b)

        # COSINE SIMILARITY DISTANCE
        cts = cupy.matmul( text_embeddings, text_embeddings[a:b].T).T

        for k in range(b-a):
            IDX = cupy.where(cts[k,]>0.7)[0]
            o = df.iloc[cupy.asnumpy(IDX)].posting_id.values
            preds.append(o)
    
    del model,text_embeddings
    gc.collect()
    return preds


In [ ]:
def combine_predictions(row):
    x = np.concatenate([row['text_predictions']])
    return ' '.join( np.unique(x))

In [ ]:
text_predictions = get_text_predictions(df, max_features = 25_000)

In [ ]:
df['text_predictions'] = text_predictions
df['matches'] = df.apply(combine_predictions, axis = 1)
df[['posting_id', 'matches']].to_csv('submission.csv', index = False)